In [1]:
# Define project information
# PROJECT_ID = "graphical-fort-306505" # @param {type:"string"}
# LOCATION = "us-central1" # @param {type:"string"}
PROJECT_ID = "wscc-dev-app-wsky" # @param {type:"string"}
LOCATION = "us-east4" # @param {type:"string"}

# Initialize Vertex AI
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [2]:
# Utils
import time
from typing import List

# Langchain
import langchain
from pydantic import BaseModel

print(f"LangChain version: {langchain.__version__}")

# Vertex AI
from google.cloud import aiplatform
from langchain.chat_models import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.schema import HumanMessage, SystemMessage

print(f"Vertex AI SDK version: {aiplatform.__version__}")

LangChain version: 0.0.229
Vertex AI SDK version: 1.33.1


In [3]:
# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)


class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

In [33]:
# LLM model
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Chat
chat = ChatVertexAI()

# Embedding
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

##Extracting text from pdf using cloud vision

In [34]:
import os
from google.cloud import vision
from google.cloud.vision_v1 import types

# adunai: we don't need this since we mount ~/.config/gcloud into container.
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/graphical-fort-306505-a3c9bca468bb.json"

# We need to update the gcp credential above with our own project API key for the Cloud vision.

In [48]:
import json
import re
from google.cloud import vision
from google.cloud import storage

import os

# Supported mime_types are: 'application/pdf' and 'image/tiff'
mime_type = 'application/pdf'

# How many pages should be grouped into each json output file.
batch_size = 10
client = vision.ImageAnnotatorClient()
feature = vision.Feature(type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

# adunai: updated this section to use WSCC project
BUCKET = 'wscc-sandbox-eai'
PREFIX = 'eai-ai-vision-pdf'
FILENAME = 'Sample_Medical-Record-part-3.pdf'
storage_client = storage.Client(project=PROJECT_ID)
gcs_bucket = storage_client.bucket(BUCKET)
pdf_blob = gcs_bucket.blob(f'{PREFIX}/input/{FILENAME}')
print('Upload PDF')
pdf_blob.upload_from_filename(FILENAME)

gcs_source = vision.GcsSource(uri=f"gs://{BUCKET}/{PREFIX}/input/{FILENAME}")

# We need to pass the input pdf file placed in the gcs which is expected by Cloud Vision API, by updating the path above under URI.

input_config = vision.InputConfig(gcs_source=gcs_source, mime_type=mime_type)

gcs_destination = vision.GcsDestination(uri=f"gs://{BUCKET}/{PREFIX}/output/")

# We also need to pass the output bucket location in which the extracted json files will be placed by the Cloud Vision API by updating above URI.

output_config = vision.OutputConfig(gcs_destination=gcs_destination, batch_size=batch_size)

async_request = vision.AsyncAnnotateFileRequest(
    features=[feature],
    input_config=input_config,
    output_config=output_config
)


print('Start annotation')
operation = client.async_batch_annotate_files(requests=[async_request])
print('Waiting for the operation to finish (this should take less than a minute)')
operation.result(timeout=120)

# Once the request has completed and the output has been
# written to GCS, we can list all the output files.

print('Gather results')
match = re.match(r'gs://([^/]+)/(.+)', f"gs://{BUCKET}/{PREFIX}/output/")
bucket_name = match.group(1)
prefix = match.group(2)
bucket = storage_client.get_bucket(bucket_name)

# List objects with the given prefix.
blob_list = list(bucket.list_blobs(prefix=prefix))
print('Output files:')
for blob in blob_list:
  print(blob.name)

Upload PDF
Start annotation
Waiting for the operation to finish (this should take less than a minute)
Gather results
Output files:
eai-ai-vision-pdf/output/output-1-to-1.json
eai-ai-vision-pdf/output/output-1-to-10.json
eai-ai-vision-pdf/output/output-11-to-20.json
eai-ai-vision-pdf/output/output-21-to-30.json
eai-ai-vision-pdf/output/output-31-to-40.json
eai-ai-vision-pdf/output/output-41-to-49.json


In [49]:
text=''
# Process all the output files from GCS.
for output in blob_list:
    json_string = output.download_as_string()
    response = json.loads(json_string)

    # Loop through responses for each page

    for page_response in response['responses']:
        annotation = page_response['fullTextAnnotation']

        # Print the text from each page
        text = text + annotation['text']


In [50]:
text

'Name: John Brown\nAccount No: 12345\nAttending Physician: Jeff T. Moore, M.D.\nConsulting Physician\nAdm Date: 11/12/1999 DOB: 09/10/44\nACTIVE ORDERS\nFTAB 500MG, CEPHALEXIN HCL MONOHYDRATE\nTAKE EVERY TWELVE HOURS\nDR THERAPY SUMARY REPORT\nTAKE ON EMPTY STOMACH-1HR AC OR 2HR PC\n***** COMMENTS,\nRTAB 10MG, HYDROCODONE 10/APAP 500\nTAKE EVERY FOUR HOURS AS NEEDED\n***** COMMENTS_____\nSTARIL, HYDROXYZINE HCL\nALLERGIES NKA\nMEROL 75MG AMP, MEPERIDINE HCL\nTAKE EVERY THREE HOURS AS NEEDED\n***** COMMENTS_____\nGIVE WITH DEMEROL\nNOT TO BE GIVEN IV\nMEDICATIONS\n***** COMMENTS,\nTAKE EVERY THREE HOURS AS NEEDED FOR\nPAIN\n"S\nDIAG:\n500MG=1TAB\nORAL\nfost of ting biover_\n10MG=1TAB\nORAL\n75MG 1.5ML\nINTRAMUSCULAR\n50MG=1ML\nINTRAMUSCULAR\nADDITIONAL DISCHARGE MEDS\npay\nGE\nHOME MED INSTRUC\nse\nAN SC\nCHECK ONE\nI REFUSED CARDS\n1 OF\nN\nN se.\nThe\nwww.\nSe\nINSTRUC\nSimp Complell\nSIGNATURE OF DISCHARGE INSTRUCTC\ny\nVERBAL INSTRUCTIONS GIVEN\n1 VERBAL INSTRUCTIONS REFUSEDAHIMA\nC

In [51]:
# split the documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
docs = text_splitter.split_text(text)
print(f"# of documents = {len(docs)}")

# of documents = 52


In [52]:
embeddings

CustomVertexAIEmbeddings(client=<vertexai.preview.language_models._PreviewTextEmbeddingModel object at 0x129012910>, model_name='textembedding-gecko', temperature=0.0, max_output_tokens=128, top_p=0.95, top_k=40, stop=None, project=None, location='us-central1', credentials=None, request_parallelism=5, max_retries=6, requests_per_minute=100, num_instances_per_batch=5)

In [53]:
# Store docs in local vectorstore as index
# it may take a while since API is rate limited
from langchain.vectorstores import Chroma

db = Chroma.from_texts(docs, embeddings)

# Sometimes we might get error for the panda version. We can ignore and try to re-run this command multiple times.

Waiting
..........

In [54]:
# Expose index to the retriever
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [55]:

# Create chain to answer questions
from langchain.chains import RetrievalQA

# Uses LLM to synthesize results from the search index.
# We use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

In [56]:
query = "What is the outcome of the ecg report?"
result = qa({"query": query})
print(result)

{'query': 'What is the outcome of the ecg report?', 'result': 'The ECG report shows that the patient has a normal sinus rhythm with nonspecific T wave abnormalities.', 'source_documents': [Document(page_content='帕帕帕关状秘书长书 URINALYSIS 开式开关册开书处长说黄英并发书区同兴戈兴师关关长否开怀忧书据我認同好性关法\n1.360 [0-43\nNORMAN\nNG/ML\nUNITS\n1.005 1.050\n5.0-9.0\nWED\nNET\n※※※并沁沁沁沁并预防并册并开派并沿浦 TUMOR MARKELVE ※泌并防并沿海河北石油供跨开4\nEUBrown, John\n55years\nMale\nRoom: TBA\nCaucasian\n40 Hz\nMeds: SURG 11/12/99\nDOB: 9/10/44\nTechnician: TB\nVent. rate\nPR interval\nQRS duration\nQT/QTc\nP-R-T axes\n25.0 mm/s\n79 bpm\n10.0 mm/mV\n174 ms\n88 ms\n360/413 ms\n44 20 79\nahatrahth\nAVR\naVL\ndVF\nSE\n10-NOV-1999 16:17:28\nNormal sinus rhythm\nNonspecific T wave abnormality\nAbnormal ECG\nB\nReferred by: Moore, J\nt\n4 by 2.5s + 1 rhythm ld\nUnconfirmed\n111\n№.4\n5\n-\nInf\nMAC 8 002B\n12SLtm v250Community General Hospital\nAnytown, USA\nRADIOLOGY REPORT\nPage 1 of 1\nEXAMINATION DESCRIPTION: Chest PA & Lateral\nName: John Brown\nDOB: 0

In [57]:
query = "What is the discharge summary of john brown?"
result = qa({"query": query})
print(result)

{'query': 'What is the discharge summary of john brown?', 'result': 'The patient was admitted with a right inguinal hernia. He underwent a right inguinal hernia repair. He tolerated the procedure well and was discharged home in good condition.', 'source_documents': [Document(page_content="Same\nPayer\nSouthern Company\nFinancial Class\nProvisional Diagnosis\nPrincipal Procedures and Secondary Procedures\nConsultations\nCommunity General Hospital\nINPATIENT REGISTRATION AND SUMMARY FORM\nDisposition of Case\nHome\nEmployer Name\nBig Company\nExpired\nTransferred to:\nRight Ing Hernia\nPrincipal Diagnosis, Secondary Diagnosis and Complications\nAnytown USA\nAdmitted By\nSwing Bed\nAutopsy\nAD12\nAttending Physician Number and Name\nJeff T. Moore\nBirth Date\nAge\n9/10/44 54\nAddress\n☐AMA\nPolicy Number\n123456789\nSupervised Living\nSex\nM\nHome\nHealth\n97\nAttending Physician\nMarital\nSt\nM\nPatient Type\nInp\nReligion\nEmployer City/State\nAnytown, US\nCity/State\nPatient Weight\nGu

In [58]:
query = "Please refer the medication page. Please tell the medication prescribed to the patient?"
result = qa({"query": query})
print(result)

{'query': 'Please refer the medication page. Please tell the medication prescribed to the patient?', 'result': 'The patient is prescribed the following medications:\nFTAB 500MG, CEPHALEXIN HCL MONOHYDRATE\nSTARIL, HYDROXYZINE HCL\nRTAB 10MG, HYDROCODONE 10/APAP 500\nMEROL 75 MG AMP, MEPERIDINE HCL', 'source_documents': [Document(page_content='below, you are acknowledging that you have received a copy of our Notice of Privacy Practices.\nPatient\'s Name: (please print).\nJoha\nBowp\nPatient, Parent, Guardian, Agenix Prown\nLivs\nWitness:\nIf other than patient, indicate relationship:\nDate:\nDate: 11.10.rime: 322\nTime:\nGuarantee of Account by Person other than Patient: I agree to accept financial responsibility for services rendered to the patient and to accept the terms of\nthe Guarantee of Account and Assignment of Insurance Benefits above.\nFinancially Responsible Party:\nDate:\nTime:\nHR3Name: John Brown\nAccount No: 12345\nAttending Physician: Jeff T. Moore, M.D.\nConsulting Phys

In [24]:
# adunai: Let's try this!
query = "What is patient's disposition of case?"
result = qa({"query": query})
print(result)

{'query': "What is patient's disposition of case?", 'result': 'The patient is going to be discharged to their home.', 'source_documents': [Document(page_content='Are you currently able to care for yourself at home?\nDo you plan to return to your home after discharge?\nWill you have someone to assist you when you leave the hospital? xes\nCURRENTLY RECEIVING\nHome Health Care\nHomemaker\nHospice\nNA\nMeals on Wheels\n●\nNA\n●\n●\n●\nCARE COORDINATION/SOCIAL SERVICE SCREENING:\nCircle as appropriate:\nNeeds\n62-6018-50-1003\nÁYes\nYes\nYes No Social Services Notified:\nCopy on Chart:\nYes No\nCopy on Chart: Yes No\nYes No\nParent(s) ☐Children\nANTICIPATED NEEDS/SERVICES UPON DISCHARGE\nFinancial\nPT/OT\nTransportation\nLong Term Care\nNA\nCardio/Pulmonary Rehab\nAdult Day Care\nSwing Bed\nHome Health\nLifeline\nFAMILY INVOLVEMENT/CARE GIVERS:\nPatient support systems\nSpouse\nName/Phone of support person:\nBEHAVIORAL/SOCIAL/COGNITIVE\nFACTOR:\nAre you currently receiving treatment for emo

In [25]:
query = "What is patient's diagnosis code?"
result = qa({"query": query})
print(result)

{'query': "What is patient's diagnosis code?", 'result': "The patient's diagnosis code is 000.", 'source_documents': [Document(page_content="Lai ( thrifty On Det Gris & W). Andy Tomast (0)\nHV k\n()\n)\n(O)\n()\n0025\n0445 fDATE:\nTEMPERATURE\nPULSE RATE\nRESP\nINTAKE AND OUTPUT\nHYGIENE\nCommunity General Hospital\nAnytown, USA\nACTIVIY\n104\n103\n102\nB/P\nPAIN RATING (0-10)\n101\n100\n99\n98\n97\n96\n140\n130\n120\n110\nGraphics Flowsheet\nAdmit\n08\n100\n90\n80\n70\n60\nWT\nSpO₂\nINTAKE\nDIET/SNACK\n%\nORAL\nIV\nBLOOD\nSHIFT\n62-6018-37-0602\n40\n30\n20\n24 HR TOTAL\nOUTPUT\nURINE/FOLEY\nBM\nHEMO DUVAL\nGAST SUCT\nEMESIS\nSHIFT TOTAL\n24 HR TOTAL\nBATHING\nORAL CARE\nPERICARE\nLOTION RUB\nREPOSITION\nEDGE OF BED\nUP IN C\nAMBULAN\n04\n196.7\nAM\n12 16 20 24\n182\n0600-1800\n20\n134/79\n0600-1800\n1800-0600\nSNACK 1\n2\n3\n1800-0600\nPM\n04\n0600-1800\n240\n887\n08\n7000\n2051\n0600-1800\n1850\nAM\n12\nV/₂\nName: John Brown\nAccount No: 12345\nAttending Physician: Jeff T. Moore, M.D

In [26]:
query = "What is patient's demographics summary?"
result = qa({"query": query})
print(result)

{'query': "What is patient's demographics summary?", 'result': "The patient's name is John Brown. The patient is 55 years old and was born on 09/10/44. The patient's gender is not specified. The patient's address is not specified. The patient's phone number is not specified. The patient's insurance provider is not specified. The patient's primary care physician is J Moore.", 'source_documents': [Document(page_content="Readiness to learn: (circle) Receptive Poor What language(s) do you read, write and understand:\nAre you still in school? Yes No Is the school and/or your teacher aware of your hospitalization? Yes No\nEnglish\nCOPING:\nDo you have concerns or fears regarding this hospitalization?No Yes If yes, explain\n☐Friend\nHospice\nSupervised Living\nMove in with Family\nN/A\nDOB: 09/10/44\nClients with no identifiable source of medical payment\nHigh Risk Obstetrical (unmarried, pregnant minors, high risk or complicated pregnancy)\nPotential or actual history of noncompliance with h

In [27]:
query = "What is patient's age as in today?"
result = qa({"query": query})
print(result)

{'query': "What is patient's age as in today?", 'result': "The patient's age is 75 years old.", 'source_documents': [Document(page_content="Readiness to learn: (circle) Receptive Poor What language(s) do you read, write and understand:\nAre you still in school? Yes No Is the school and/or your teacher aware of your hospitalization? Yes No\nEnglish\nCOPING:\nDo you have concerns or fears regarding this hospitalization?No Yes If yes, explain\n☐Friend\nHospice\nSupervised Living\nMove in with Family\nN/A\nDOB: 09/10/44\nClients with no identifiable source of medical payment\nHigh Risk Obstetrical (unmarried, pregnant minors, high risk or complicated pregnancy)\nPotential or actual history of noncompliance with health care plan\nClients admitted with high risk diagnosis (example: COPD, CHF, Diabetes)\nSuspected victim of abuse (see below)\nFor any 'Yes' answers above, please notify Care Coordination Department and/or Social Service Department\nSignature\nONeighbor How many hours/day?\nDate

In [28]:
query = "What is patient's medications summary?"
result = qa({"query": query})
print(result)

{'query': "What is patient's medications summary?", 'result': "The patient is taking Goody's and Ginge.", 'source_documents': [Document(page_content="24. Stamp plate/fabe attached to chart\n25. IV pump if required for special drip / pediatric patients (HAF, Heparin, etc.)\n26. Checklist reviewed on unit by:\n27. Patient accepted for O.R. transport by:\nDate:\nSIGNATURE\nCommunity General Hospital\nAnytown, USA\n000000000\nPRE-OPERATIVE\nCHECKLIST\n/sm\nINITIAL\nINITIAL\nGratt 115\nTime:\nINITIAL\nDisposition\nI wife\nRN.\nSIGNATURE\nCHECK AS\nAPPROPRIATE\nSIGNATURE\n| □□□□□□□□□□□\nYES\nKatte Craft Ro\nSIGNATURE\n0080000000309900µ×¤×3\n100720020000000000 9970**-**\n--->¯¯¯¯¯¯¯000003\nIntact N/A\nINITIALS\nsm\nКс\nsm\nIm\nn\n&m\n$\n2\nsm\nsm\nR\nKC\nsm\nKu\nkc\n+6\nsm\nкс\nкс\nКс\nEm\nRe\nK\nAKC\nINITIAL\nINITIAL\nINITIALName: John Brown\nAccount No: 12345\nAttending Physician: Jeff T. Moore, M.D.\nConsulting Physician\nAdm Date: 11/12/1999 DOB: 09/10/44\nSYSTEM REVIEW\nPATIENT HISTORY\n

In [29]:
query = "What is patient's demographics as json document?"
result = qa({"query": query})
print(result)

{'query': "What is patient's demographics as json document?", 'result': '{\n  "name": "John Brown",\n  "account_number": "12345",\n  "admitted_date": "11/12/1999",\n  "attending_physician": "Jeff T. Moore, M.D.",\n  "consulting_physician": ""\n}', 'source_documents': [Document(page_content='Are you currently able to care for yourself at home?\nDo you plan to return to your home after discharge?\nWill you have someone to assist you when you leave the hospital? xes\nCURRENTLY RECEIVING\nHome Health Care\nHomemaker\nHospice\nNA\nMeals on Wheels\n●\nNA\n●\n●\n●\nCARE COORDINATION/SOCIAL SERVICE SCREENING:\nCircle as appropriate:\nNeeds\n62-6018-50-1003\nÁYes\nYes\nYes No Social Services Notified:\nCopy on Chart:\nYes No\nCopy on Chart: Yes No\nYes No\nParent(s) ☐Children\nANTICIPATED NEEDS/SERVICES UPON DISCHARGE\nFinancial\nPT/OT\nTransportation\nLong Term Care\nNA\nCardio/Pulmonary Rehab\nAdult Day Care\nSwing Bed\nHome Health\nLifeline\nFAMILY INVOLVEMENT/CARE GIVERS:\nPatient support s

In [30]:
query = "Please give json document with patients dob,name, address, gender, phone number?"
result = qa({"query": query})
print(result)

{'query': 'Please give json document with patients dob,name, address, gender, phone number?', 'result': "The patient's name is John Brown. Their date of birth is 09/10/44. Their address is 2222222. Their gender is not present. Their phone number is not present.", 'source_documents': [Document(page_content='Ordering Physician: Jeff T. Moore\nPathologist: Sally Johnson, M.D.\nLocation: 3W 0328 P\nGross Description\nReceived in formalin and labeled hernia sac is a grossly identifiable encapsulated\nfragment of yellow fibroadipose tissue that measures 4.5 x 1.5 x 0.6 cm in widest\ndimensions. The specimen is cross sectioned which reveals a surface that is\nhomogeneously balanced and encapsulated with a thin tan-brown membranous material.\nNo ulceration, pigmentation or nodular abnormalities can be grossly identified.\nRepresentative portions submitted in one cassette.\nDiagnosis\nSoft tissue inguinal region: Hernia sac containing hemorrhage and areas of fibrosis,\nnegative for malignancy.\

In [31]:
query = "what is patients dob,name, address, gender, phone number? Please provide it in json format"
result = qa({"query": query})
print(result)

{'query': 'what is patients dob,name, address, gender, phone number? Please provide it in json format', 'result': "The patient's name is John Brown. Their DOB is 09/10/44. Their address is unknown. Their gender is unknown. Their phone number is unknown.", 'source_documents': [Document(page_content="Readiness to learn: (circle) Receptive Poor What language(s) do you read, write and understand:\nAre you still in school? Yes No Is the school and/or your teacher aware of your hospitalization? Yes No\nEnglish\nCOPING:\nDo you have concerns or fears regarding this hospitalization?No Yes If yes, explain\n☐Friend\nHospice\nSupervised Living\nMove in with Family\nN/A\nDOB: 09/10/44\nClients with no identifiable source of medical payment\nHigh Risk Obstetrical (unmarried, pregnant minors, high risk or complicated pregnancy)\nPotential or actual history of noncompliance with health care plan\nClients admitted with high risk diagnosis (example: COPD, CHF, Diabetes)\nSuspected victim of abuse (see 

In [62]:
query = "Please provide list of medicatiosn patient is taking?"
result = qa({"query": query})
print(result)

{'query': 'Please provide list of medicatiosn patient is taking?', 'result': "Goody' Ginger\n[eod]", 'source_documents': [Document(page_content="24. Stamp plate/fabe attached to chart\n25. IV pump if required for special drip / pediatric patients (HAF, Heparin, etc.)\n26. Checklist reviewed on unit by:\n27. Patient accepted for O.R. transport by:\nDate:\nSIGNATURE\nCommunity General Hospital\nAnytown, USA\n000000000\nPRE-OPERATIVE\nCHECKLIST\n/sm\nINITIAL\nINITIAL\nGratt 115\nTime:\nINITIAL\nDisposition\nI wife\nRN.\nSIGNATURE\nCHECK AS\nAPPROPRIATE\nSIGNATURE\n| □□□□□□□□□□□\nYES\nKatte Craft Ro\nSIGNATURE\n0080000000309900µ×¤×3\n100720020000000000 9970**-**\n--->¯¯¯¯¯¯¯000003\nIntact N/A\nINITIALS\nsm\nКс\nsm\nIm\nn\n&m\n$\n2\nsm\nsm\nR\nKC\nsm\nKu\nkc\n+6\nsm\nкс\nкс\nКс\nEm\nRe\nK\nAKC\nINITIAL\nINITIAL\nINITIALName: John Brown\nAccount No: 12345\nAttending Physician: Jeff T. Moore, M.D.\nConsulting Physician\nAdm Date: 11/12/1999 DOB: 09/10/44\nSYSTEM REVIEW\nPATIENT HISTORY\nMEDIC

In [33]:
query = "Please provide list of allergies?"
result = qa({"query": query})
print(result)

{'query': 'Please provide list of allergies?', 'result': 'The patient has no known allergies.', 'source_documents': [Document(page_content='Night Sweats: Yes No Increased Fatigue: Yes No Recent TB skin test: Yes No, if yes reactor: Yes No\n*Respiratory therapy notified for abnormal assessment findings: Yes DAR note abnormal finding)\nLLL: Clear Adventitious\n*Any chewing or swallowing problems:\nHistory of cancer, diabetes, or renal disease\nHas > 3 alcohol drinks/day (Women) > 4 (Men)\nRecent weight gain\nGASTROINTESTINAL/NUTRITION\nBowel Sounds: Normal Hypoactive Hyperactive Absent\nAbdomen: Soft Firm Hard\nBM: Continent Incontinent Constipation Diarrhea Bloody Mucus Other: Pos BS in all 4 fund.\nLaxative Usage:\nLast BM:\n11/11/99\n(R) Pedal: Present Absent Noted with Doppler\n(L) Pedal: Present Absent Noted with Doppler\nYes\nYes\nYes\nNo\nYes No\nOpen wounds, decubitus ulcers or trauma\nYes No\n*Is on a special diet/or special diet ordered\nYes No\n*If any "yes" answers, registere

In [61]:
query = "please summrize allergies?"
result = qa({"query": query})
print(result)

{'query': 'please summrize allergies?', 'result': "I don't know\n[eod]", 'source_documents': [Document(page_content='Night Sweats: Yes No Increased Fatigue: Yes No Recent TB skin test: Yes No, if yes reactor: Yes No\n*Respiratory therapy notified for abnormal assessment findings: Yes DAR note abnormal finding)\nLLL: Clear Adventitious\n*Any chewing or swallowing problems:\nHistory of cancer, diabetes, or renal disease\nHas > 3 alcohol drinks/day (Women) > 4 (Men)\nRecent weight gain\nGASTROINTESTINAL/NUTRITION\nBowel Sounds: Normal Hypoactive Hyperactive Absent\nAbdomen: Soft Firm Hard\nBM: Continent Incontinent Constipation Diarrhea Bloody Mucus Other: Pos BS in all 4 fund.\nLaxative Usage:\nLast BM:\n11/11/99\n(R) Pedal: Present Absent Noted with Doppler\n(L) Pedal: Present Absent Noted with Doppler\nYes\nYes\nYes\nNo\nYes No\nOpen wounds, decubitus ulcers or trauma\nYes No\n*Is on a special diet/or special diet ordered\nYes No\n*If any "yes" answers, registered dietician or dietary 